In [ ]:
from bs4 import BeautifulSoup
import re
def load_and_clean_data(file_path) :
    with open(file_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
        text = soup.get_text()
        # {% ... %} 같은 Django 템플릿 태그 제거
        text = re.sub(r"{%.*?%}", "", text)
        return text
print(load_and_clean_data("templates/product/ai_computing/AI_Train_이용_방법.html"))



 
AI Train 이용 방법    

AI Train 이용 방법
서비스 신청


    AI Train는 D1 플랫폼의 DX-DCN-CJ 존에서만 제공됩니다.
    상품 신청 시 최초 1회는 신규 존(DX-DCN-CJ)을 생성하는 과정이 필요합니다.
    콘솔 상단 'Platform' 메뉴를 클릭합니다.


화면에서 D1 플랫폼과 그 하위의 DX-DCN-CJ존을 선택한 후 '신청' 버튼을 클릭합니다.

D1 플랫폼의 DCN-CJ 존 최초 신청을 위한 화면으로 넘어가고, 'DCN-CJ 신청하기' 버튼을 클릭하여 존을 생성합니다.
    존 생성은 통상 약  5분이 소요되며, 존 생성이 완료된 후 다음 단계를 진행합니다.

좌측 메뉴 상단의 ‘+All Services’ 에서 “서비스 신청” 버튼을 클릭 후 AI Train상품에 대해 “신청하기”를 클릭해서  상품을 신청합니다.

이제 좌측 서비스 탭에서 AI Train를 이용할 수 있습니다.





# RAG 적용 (기본 방식)

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser
import os

# 1. 기존에 만든 chroma_db 경로
persist_directory = "./chroma_db"

# 2. 임베딩 모델 (기존과 동일)
embedding_model = OpenAIEmbeddings()

# 3. 저장된 Chroma DB 불러오기
vectordb = Chroma(
    embedding_function=embedding_model,
    persist_directory=persist_directory,
    collection_name="my_db"
)

# 4. Retriever 준비
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# 5. LLM 준비
llm = ChatOpenAI(temperature=0)

# 6. 프롬프트 템플릿 작성 (검색 결과를 포함해 질문에 답변하도록 유도)
prompt_template = """
다음은 사용자가 질문한 내용과 관련된 문서입니다:
{context}

질문: {question}

이 정보를 참고하여 자세하고 친절하게 답변해 주세요.
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# 7. LLM 체인 생성 (입력: context + question, 출력: 답변)
llm_chain = LLMChain(llm=llm, prompt=prompt, output_parser=StrOutputParser())

# 8. 사용자 질문에 대해 검색 + LLM 실행 함수
def answer_question(query: str):
    # 8-1. Retriever로 관련 문서 검색
    docs = retriever.get_relevant_documents(query)
    
    # 8-2. 문서 내용 합치기 (context 생성)
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # 8-3. LLM 체인 실행 (context + 질문)
    response = llm_chain.run({"context": context, "question": query})
    
    return response, docs

# 9. 질문 입력 및 답변 출력 (예시)
if __name__ == "__main__":
    user_query = input("질문을 입력하세요: ")
    answer, source_docs = answer_question(user_query)

    print("\n=== 답변 ===")
    print(answer)
    print("\n=== 참고 문서 ===")
    for i, doc in enumerate(source_docs, 1):
        print(f"[{i}] {doc.metadata.get('source', '출처 없음')}")
        print(doc.page_content[:500])  # 문서 내용 앞부분 일부 출력
        print("---")

C:\Users\tnstn\AppData\Local\Temp\ipykernel_21432\132586360.py:15: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()
C:\Users\tnstn\AppData\Local\Temp\ipykernel_21432\132586360.py:18: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(
C:\Users\tnstn\AppData\Local\Temp\ipykernel_21432\132586360.py:28: LangChainDeprecationWarning: The class `Cha


=== 답변 ===
DBaaS for MySQL 8 서비스는 최신 MySQL 데이터베이스 엔진(MySQL 8.0.33)을 기반으로 한 관리형 DB 서비스를 제공합니다. 이를 통해 사용자는 MySQL Native한 클러스터 구조를 자동으로 구성하여 고 가용성과 서비스 연속성을 보장 받을 수 있습니다. 또한 UI를 통해 손쉽게 MySQL 구조를 변경할 수 있고, 다양한 관리 기능을 통해 안정적으로 DB 서비스를 이용할 수 있습니다. 따라서 DBaaS for MySQL 8은 사용자에게 편리하고 안정적인 MySQL 데이터베이스를 제공합니다.

=== 참고 문서 ===
[1] templates/product/db\DBaaS for MySQL 8 FAQ.html
DBaaS for MySQL 8 FAQ    


FAQ
FAQ

DBaaS for MySQL 8은 어떤 데이터베이스를 제공하나요 ?

DBaaS for MySQL 8 서비스는 최신 MySQL 데이터베이스 엔진(MySQL 8.0.33) 기반의 관리형 DB 서비스를 제공합니다.

DBaaS for MySQL 8을 외부에서도 사용이 가능한가요?

외부 접근 가능 (Access 설정)에 대한 옵션이 제공 되며, 해당 기능을 통해 외부에서도 접근 가능 합니다.

DBaaS for MySQL 8 서비스는 어느 위치(존)에서 사용이 가능한가요?

현재, kt cloud의 DX-M1, DX-Central 존과  kt gcloud의 DX-G, DX-G-YS 존에서 사용 가능합니다.

DBaaS for MySQL 8에서 제공하는 DB 구성 방식(구조)은 어떤 종류가 있나요?
---
[2] templates/product/db\DBaaS for MySQL 8 상품 개요.html
DBaaS for MySQL 8 상품 개요    


상품 개요
목적/용도


    DBaaS for MySQL 8은 최신 MySQL 데이터베이스를 편리하게 배포하고 운영할 수 있는 서비스입니다.
    특히, MySQL Native한 클러스터 구조를 

# RAG 적용 (RetrievalQA로 LANGCHAIN 만들기)

In [11]:
# 필요한 패키지 설치 (처음 한 번만)
# !pip install langchain chromadb openai

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

import os

# 1. 기존에 만든 chroma_db 경로
persist_directory = "./chroma_db"

# 2. 임베딩 모델 (기존과 동일해야 함)
embedding_model = OpenAIEmbeddings()

# 3. 저장된 Chroma DB 불러오기
vectordb = Chroma(
    embedding_function=embedding_model,
    persist_directory=persist_directory,
    collection_name="my_db"  # 기존에 만든 컬렉션 이름
)

# 5. RAG용 Chain 구성
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 2})
llm = ChatOpenAI(temperature=0)  # OpenAI API 호출 (GPT-4 등)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 6. 질문 입력 & 답변 실행
query = input()
result = qa_chain(query)

print("질문:")
print(query)
print("답변:")
print(result['result'])
print("\n참고 문서:")
for doc in result['source_documents']:
    print("-", doc.page_content)

질문:
Django는 뭐야?
답변:
Django는 Python으로 작성된 오픈 소스 웹 애플리케이션 프레임워크입니다. Django는 웹 애플리케이션을 빠르고 쉽게 개발할 수 있도록 도와주는 많은 기능과 도구를 제공합니다. 데이터베이스와의 상호작용, URL 라우팅, 템플릿 엔진, 보안 기능 등을 포함한 다양한 기능을 제공하여 개발자들이 효율적으로 웹 애플리케이션을 구축할 수 있도록 도와줍니다.

참고 문서:
- DBaaS for MySQL 8 FAQ    


FAQ
FAQ

DBaaS for MySQL 8은 어떤 데이터베이스를 제공하나요 ?

DBaaS for MySQL 8 서비스는 최신 MySQL 데이터베이스 엔진(MySQL 8.0.33) 기반의 관리형 DB 서비스를 제공합니다.

DBaaS for MySQL 8을 외부에서도 사용이 가능한가요?

외부 접근 가능 (Access 설정)에 대한 옵션이 제공 되며, 해당 기능을 통해 외부에서도 접근 가능 합니다.

DBaaS for MySQL 8 서비스는 어느 위치(존)에서 사용이 가능한가요?

현재, kt cloud의 DX-M1, DX-Central 존과  kt gcloud의 DX-G, DX-G-YS 존에서 사용 가능합니다.

DBaaS for MySQL 8에서 제공하는 DB 구성 방식(구조)은 어떤 종류가 있나요?
- DBaaS for PostgreSQL FAQ    


FAQ
FAQ

DBaaS for PostgreSQL은 어떤 데이터베이스를 제공하나요?

DBaaS for PostgreSQL은 PostgreSQL 데이터베이스 기반의 관리형 DB 서비스를 제공합니다.

DBaaS for PostgreSQL을 외부에서도 사용이 가능한가요?

외부 접근 가능 (Access 설정)에 대한 옵션이 제공 되며, 해당 기능을 통해 외부에서도 접근 가능 합니다.

DBaaS for PostgreSQL은 어느 위치(존)에서 사용이 가능한가요?

현재, kt cloud의 DX-M1, DX-Central 존과  kt gclo

# RAG 미적용

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

# 1. LLM 준비
llm = ChatOpenAI(temperature=0)  # GPT-4 등

# 2. 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_template(
    "너는 친절한 AI야. 다음 질문에 대답해줘: {question}"
)

# 3. 출력 파서 (문자열로 결과 반환)
output_parser = StrOutputParser()

# 4. LangChain 구성 (프롬프트 → LLM → 파서)
chain = prompt | llm | output_parser

# 5. 질문 입력 및 실행
query = input("질문을 입력하세요: ")
result = chain.invoke({"question": query})

print("\n질문:")
print(query)
print("\n답변:")
print(result)


질문:
VPN 설정 작업 소요 시간은 어느정도 걸리나요?

답변:
VPN 설정 작업 소요 시간은 다양하며, 사용하는 VPN 서비스나 설정하는 환경에 따라 다를 수 있습니다. 일반적으로는 몇 분에서 30분 정도의 시간이 소요될 수 있습니다. 그러나 복잡한 설정이 필요한 경우에는 더 많은 시간이 소요될 수도 있습니다. 설정하는 방법에 대한 자세한 안내는 해당 VPN 서비스의 공식 웹사이트나 고객 지원팀에 문의하시는 것이 좋습니다.


In [ ]:
from langgraph.graph import StateGraph
from typing_extensions import TypedDict

# graph 객체 생성(선언)
# simple_graph 상 데이터 전달에 사용할 객체 -> State 클래스
class State(TypedDict):
    input: str    # 사용자 input data
    result: str   # Node의 result data
graph = StateGraph(State)

In [ ]:
def is_new_session(state: State) -> bool:
    if state["input"] == "exit" :


In [16]:
from IPython.display import Image, display

try:
    display(
        Image(
            graph.get_graph().draw_mermaid_png()
        )
    )
except:
    pass